In [1]:
import pandas as pd

In [3]:
data = pd.read_csv('../data/play_by_play.txt', delimiter = "\t")
data

,Game_id,Event_Num,Event_Msg_Type,Period,WC_Time,PC_Time,Action_Type,Option1,Option2,Option3,Team_id,Person1,Person2,Team_id_type
0,021fd159b55773fba8157e2090fe0fe2,0,12,1,546427,7200,0,0,0,0,1473d70e5646a26de3c52aa1abd85b1f,6bcf6c1f8c373d25fca1579bc4464a91,6bcf6c1f8c373d25fca1579bc4464a91,0
1,021fd159b55773fba8157e2090fe0fe2,1,10,1,546495,7200,0,0,0,0,012059d397c0b7e5a30a5bb89c0b075e,89706b99ddd00dc05d37ef5cafc04276,307beab25b1021a548b4a47550bc4b25,2
2,021fd159b55773fba8157e2090fe0fe2,2,2,1,546665,7050,1,3,0,0,012059d397c0b7e5a30a5bb89c0b075e,cec898a1d355dbfbad8c760615fde1af,6bcf6c1f8c373d25fca1579bc4464a91,2
3,021fd159b55773fba8157e2090fe0fe2,3,4,1,546714,6960,0,0,0,0,012059d397c0b7e5a30a5bb89c0b075e,307beab25b1021a548b4a47550bc4b25,6bcf6c1f8c373d25fca1579bc4464a91,2
4,021fd159b55773fba8157e2090fe0fe2,6,6,1,546886,6920,4,0,0,0,cff694c8186a4bd377de400e4f60fe47,c00264c3114d23bac482e9de50fb7d28,89706b99ddd00dc05d37ef5cafc04276,3
5,021fd159b55773fba8157e2090fe0fe2,7,5,1,546887,6920,5,0,0,0,cff694c8186a4bd377de400e4f60fe47,c00264c3114d23bac482e9de50fb7d28,6bcf6c1f8c373d25fca1579bc4464a91,3
6,021fd159b55773fba8157e2090fe0fe2,8,6,1,547110,6820,1,0,0,0,012059d397c0b7e5a30a5bb89c0b075e,57bbd7e30bc694aeee9ee40c583e6811,89706b99ddd00dc05d37ef5cafc04276,2
7,021fd159b55773fba8157e2090fe0fe2,9,1,1,547220,6740,49,2,0,0,012059d397c0b7e5a30a5bb89c0b075e,a99f44bbff39e352191a870e17f04537,881f83d2dee3f18c7d1751659406144e,2
8,021fd159b55773fba8157e2090fe0fe2,10,1,1,547395,6580,1,2,0,0,cff694c8186a4bd377de400e4f60fe47,57bbd7e30bc694aeee9ee40c583e6811,c00264c3114d23bac482e9de50fb7d28,3
9,021fd159b55773fba8157e2090fe0fe2,12,2,1,547524,6450,1,3,0,0,012059d397c0b7e5a30a5bb89c0b075e,881f83d2dee3f18c7d1751659406144e,6bcf6c1f8c373d25fca1579bc4464a91,2
